# GERADOR DE POEMAS COM IA E FEEDBACK

## Objetivo do Sistema

Este projeto implementa um **sistema completo de geração de poemas** usando IA generativa via API, com as seguintes funcionalidades:

- **Geração de poemas**: Uso de LLM (Large Language Model) via API
- **Feedback do usuário**: Coleta estruturada de avaliação do poema
- **Novo Chat vs Chat Contínuo**: Diferenciação entre iniciar nova conversa ou continuar com histórico
- **Interface interativa**: Widgets no Jupyter para experiência intuitiva
- **Melhoria automática**: Refinamento de prompts baseado no feedback

## Responsabilidades

- **Grupo 1**: Integração com LLM e chamadas à API
- **Grupo 2**: Interface interativa com widgets
- **Grupo 3**: Sistema de feedback e refinamento de prompts

---

In [ ]:
!pip install gradio==5.40.0
!pip install -U -q "google-genai"

### SISTEMA COMPLETO DE FEEDBACK (Grupo 3) - INTEGRADO COM HISTÓRICO

In [ ]:
# Importações necessárias para o sistema avançado
import difflib
from collections import Counter

# 1. VALIDAÇÃO E MAPEAMENTO INTELIGENTE DE REFINAMENTOS
def mapear_alvo_inteligente(alvo, poema_atual):
    """
    Mapeia descrições naturais (ex: "terceira estrofe") para texto real do poema.
    
    Args:
        alvo (str): Descrição do alvo (ex: "primeira linha", "segunda estrofe")
        poema_atual (str): Poema atual
    
    Returns:
        str: Texto real encontrado no poema ou alvo original
    """
    alvo_lower = alvo.lower().strip()
    linhas = [linha.strip() for linha in poema_atual.split('\n') if linha.strip()]
    
    # Separar em estrofes (grupos de linhas separadas por linha vazia)
    estrofes = []
    estrofe_atual = []
    
    for linha in poema_atual.split('\n'):
        if linha.strip():
            estrofe_atual.append(linha.strip())
        else:
            if estrofe_atual:
                estrofes.append('\n'.join(estrofe_atual))
                estrofe_atual = []
    if estrofe_atual:
        estrofes.append('\n'.join(estrofe_atual))
    
    # Mapeamento por posição numérica
    if 'primeira' in alvo_lower and 'linha' in alvo_lower:
        return linhas[0] if linhas else alvo
    elif 'segunda' in alvo_lower and 'linha' in alvo_lower:
        return linhas[1] if len(linhas) > 1 else alvo
    elif 'terceira' in alvo_lower and 'linha' in alvo_lower:
        return linhas[2] if len(linhas) > 2 else alvo
    elif 'última' in alvo_lower and 'linha' in alvo_lower:
        return linhas[-1] if linhas else alvo
    
    # Mapeamento por estrofe
    elif 'primeira' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[0] if estrofes else alvo
    elif 'segunda' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[1] if len(estrofes) > 1 else alvo
    elif 'terceira' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[2] if len(estrofes) > 2 else alvo
    elif 'última' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[-1] if estrofes else alvo
    
    # Mapeamento por números ordinais
    for i, num in enumerate(['primeiro', 'segunda', 'terceiro', 'quarto', 'quinto'], 1):
        if num in alvo_lower:
            if 'linha' in alvo_lower and len(linhas) >= i:
                return linhas[i-1]
            elif 'estrofe' in alvo_lower and len(estrofes) >= i:
                return estrofes[i-1]
    
    # Se não encontrou mapeamento, retorna o alvo original
    return alvo

def validar_refinamento(refinamento, poema_atual):
    """
    Valida se o refinamento faz sentido no contexto do poema.
    Agora com mapeamento inteligente de alvos.
    
    Args:
        refinamento (dict): {'acao': str, 'alvo': str, 'novo_conteudo': str}
        poema_atual (str): Poema atual
    
    Returns:
        dict: {'valido': bool, 'erro': str (se inválido), 'alvo_mapeado': str}
    """
    alvo_original = refinamento['alvo']
    acao = refinamento['acao']
    
    # Mapear alvo para texto real
    alvo_mapeado = mapear_alvo_inteligente(alvo_original, poema_atual)
    
    # Verifica se o alvo existe (para Editar e Remover)
    if acao in ['Editar', 'Remover']:
        # Primeiro tenta encontrar o alvo mapeado
        if alvo_mapeado.lower() in poema_atual.lower():
            # Encontrou o texto mapeado
            pass
        else:
            # Tenta busca por palavras-chave do alvo original
            palavras_alvo = alvo_original.lower().split()
            poema_lower = poema_atual.lower()
            
            encontrado = any(palavra in poema_lower for palavra in palavras_alvo if len(palavra) > 3)
            
            if not encontrado:
                return {
                    'valido': False,
                    'erro': f"Alvo '{alvo_original}' não encontrado no poema. Tente ser mais específico ou copiar o texto exato.",
                    'alvo_mapeado': alvo_mapeado
                }
    
    # Verifica se há novo conteúdo quando necessário
    if acao in ['Adicionar', 'Editar']:
        if not refinamento.get('novo_conteudo'):
            return {
                'valido': False,
                'erro': f"Ação '{acao}' precisa de novo conteúdo",
                'alvo_mapeado': alvo_mapeado
            }
    
    return {'valido': True, 'alvo_mapeado': alvo_mapeado}

# 2. DETECÇÃO DE CONFLITOS
def detectar_conflitos(refinamentos):
    """
    Detecta conflitos entre refinamentos.
    
    Args:
        refinamentos (list): Lista de refinamentos
    
    Returns:
        list: Lista de conflitos detectados
    """
    conflitos = []
    
    for i, ref1 in enumerate(refinamentos):
        for j, ref2 in enumerate(refinamentos[i+1:], start=i+1):
            # Mesmo alvo com ações diferentes
            if ref1['alvo'].lower() == ref2['alvo'].lower():
                if ref1['acao'] != ref2['acao']:
                    conflitos.append({
                        'tipo': 'mesmo_alvo_acoes_diferentes',
                        'indices': [i, j],
                        'descricao': f"Refinamentos {i+1} e {j+1} têm o mesmo alvo com ações diferentes",
                        'sugestao': 'Mesclar em um único refinamento ou remover um deles'
                    })
            
            # Remover seguido de editar
            if ref1['acao'] == 'Remover' and ref2['acao'] == 'Editar':
                if ref1['alvo'].lower() in ref2['alvo'].lower():
                    conflitos.append({
                        'tipo': 'editar_apos_remover',
                        'indices': [i, j],
                        'descricao': f"Refinamento {j+1} tenta editar algo que será removido",
                        'sugestao': 'Remover o refinamento de edição ou mudar a ordem'
                    })
    
    return conflitos

# 3. PRIORIZAÇÃO DE REFINAMENTOS
def priorizar_refinamentos(refinamentos):
    """
    Ordena refinamentos na ordem ideal: Remover → Editar → Adicionar
    
    Args:
        refinamentos (list): Lista de refinamentos
    
    Returns:
        list: Refinamentos ordenados
    """
    prioridades = {
        'Remover': 1,
        'Editar': 2,
        'Adicionar': 3
    }
    
    return sorted(refinamentos, key=lambda r: prioridades.get(r['acao'], 999))

# 4. TEMPLATES ESPECIALIZADOS
TEMPLATES_REFINAMENTO = {
    'Editar': """POEMA ORIGINAL:
{poema}

INSTRUÇÃO MUITO ESPECÍFICA: 
Você deve EDITAR APENAS a parte "{alvo}" para ficar "{novo_conteudo}".

REGRAS OBRIGATÓRIAS:
1. COPIE o poema original EXATAMENTE como está
2. Modifique SOMENTE a parte mencionada: "{alvo}"
3. TODO o resto deve permanecer IDÊNTICO palavra por palavra
4. NÃO reescreva o poema inteiro - apenas ajuste a parte específica
5. Mantenha quebras de linha, pontuação e formatação originais
6. Responda APENAS com o poema modificado, sem comentários

IMPORTANTE: É uma pequena modificação, não um novo poema!""",
    
    'Adicionar': """POEMA ORIGINAL:
{poema}

INSTRUÇÃO MUITO ESPECÍFICA:
Você deve ADICIONAR "{novo_conteudo}" na posição "{alvo}".

REGRAS OBRIGATÓRIAS:
1. MANTENHA todo o poema original exatamente igual
2. ADICIONE o novo conteúdo na posição indicada
3. NÃO modifique nada que já existe
4. Integre naturalmente sem quebrar o fluxo
5. Preserve formatação, quebras de linha e estilo
6. Responda APENAS com o poema completo (original + adição)

IMPORTANTE: É uma adição ao poema existente, não um novo poema!""",
    
    'Remover': """POEMA ORIGINAL:
{poema}

INSTRUÇÃO MUITO ESPECÍFICA:
Você deve REMOVER completamente a parte: "{alvo}".

REGRAS OBRIGATÓRIAS:
1. MANTENHA todo o resto do poema exatamente igual
2. REMOVA apenas o que foi especificado: "{alvo}"
3. Ajuste transições se necessário para manter fluidez
4. NÃO reescreva outras partes
5. Preserve formatação e estilo original
6. Responda APENAS com o poema sem a parte removida

IMPORTANTE: É uma remoção específica, não um novo poema!"""
}

def gerar_prompt_refinamento(refinamento, poema_atual):
    """
    Gera prompt otimizado baseado no tipo de refinamento.
    Usa estratégia de preservação forçada para evitar reescrita completa.
    
    Args:
        refinamento (dict): Refinamento a aplicar
        poema_atual (str): Poema atual
    
    Returns:
        str: Prompt formatado
    """
    acao = refinamento['acao']
    alvo = refinamento['alvo']
    novo_conteudo = refinamento.get('novo_conteudo', '')
    
    # Estratégia mais agressiva para forçar preservação
    if acao == 'Editar':
        # Quebra o poema em linhas para análise
        linhas = poema_atual.split('\n')
        linhas_numeradas = '\n'.join([f"LINHA {i+1}: {linha}" for i, linha in enumerate(linhas)])
        
        prompt = f"""TAREFA CRÍTICA DE EDIÇÃO PRECISA:

POEMA ORIGINAL COM NUMERAÇÃO (PRESERVE EXATAMENTE):
{linhas_numeradas}

MODIFICAÇÃO SOLICITADA:
- Encontre e modifique apenas: "{alvo}"
- Aplicar mudança: "{novo_conteudo}"

INSTRUÇÕES OBRIGATÓRIAS:
1. COPIE todas as linhas exatamente como estão
2. Modifique SOMENTE a parte que contém "{alvo}"
3. Mantenha numeração e estrutura idênticas
4. Se "{alvo}" se refere a uma estrofe/linha específica, modifique apenas ela
5. TODO o resto deve ser uma CÓPIA EXATA do original

FORMATO DE RESPOSTA: Retorne apenas o poema modificado (sem numeração), preservando quebras de linha originais.

CRÍTICO: Esta é uma pequena correção, não um novo poema!"""

    elif acao == 'Adicionar':
        prompt = f"""TAREFA DE ADIÇÃO CONTROLADA:

POEMA ORIGINAL (MANTENHA INTACTO):
{poema_atual}

ADIÇÃO SOLICITADA:
- Posição: {alvo}
- Conteúdo a adicionar: "{novo_conteudo}"

INSTRUÇÕES:
1. PRESERVE todo o poema original PALAVRA POR PALAVRA
2. Adicione o novo conteúdo na posição indicada
3. Mantenha formatação e quebras de linha originais
4. NÃO modifique nenhuma palavra existente

RETORNE: O poema original + a adição (sem comentários)"""

    else:  # Remover
        prompt = f"""TAREFA DE REMOÇÃO ESPECÍFICA:

POEMA ORIGINAL:
{poema_atual}

REMOVER: "{alvo}"

INSTRUÇÕES:
1. MANTENHA tudo igual exceto a parte a ser removida
2. Remova apenas "{alvo}" especificamente
3. Ajuste transições se necessário
4. Preserve formatação original

RETORNE: Poema sem a parte removida (sem comentários)"""
    
    return prompt

# 5. AVALIAÇÃO DE RESULTADOS
def avaliar_refinamento(poema_antes, poema_depois, refinamento):
    """
    Avalia se o refinamento foi aplicado com sucesso.
    
    Args:
        poema_antes (str): Poema antes do refinamento
        poema_depois (str): Poema depois do refinamento
        refinamento (dict): Refinamento aplicado
    
    Returns:
        dict: Métricas de avaliação
    """
    # Calcula diferenças
    diff = list(difflib.ndiff(poema_antes.splitlines(), poema_depois.splitlines()))
    mudancas = [linha for linha in diff if linha.startswith('+ ') or linha.startswith('- ')]
    
    # Análise básica
    resultado = {
        'mudancas_detectadas': len(mudancas) > 0,
        'num_linhas_mudadas': len(mudancas),
        'tipo_refinamento': refinamento['acao'],
        'alvo': refinamento['alvo']
    }
    
    # Verificação específica por tipo
    alvo_lower = refinamento['alvo'].lower()
    poema_depois_lower = poema_depois.lower()
    
    if refinamento['acao'] == 'Remover':
        # Sucesso se o alvo NÃO está mais presente
        resultado['sucesso'] = alvo_lower not in poema_depois_lower
        resultado['motivo'] = 'Alvo removido' if resultado['sucesso'] else 'Alvo ainda presente'
        
    elif refinamento['acao'] == 'Adicionar':
        # Sucesso se o novo conteúdo FOI adicionado
        novo_conteudo_lower = refinamento.get('novo_conteudo', '').lower()
        resultado['sucesso'] = novo_conteudo_lower in poema_depois_lower
        resultado['motivo'] = 'Conteúdo adicionado' if resultado['sucesso'] else 'Conteúdo não encontrado'
        
    else:  # Editar
        # Sucesso se houve mudanças
        resultado['sucesso'] = len(mudancas) > 0
        resultado['motivo'] = 'Poema modificado' if resultado['sucesso'] else 'Nenhuma mudança detectada'
    
    return resultado


# 6. FUNÇÃO PRINCIPAL DE PROCESSAMENTO
def processar_refinamentos(refinamentos, poema_atual):
    """
    Pipeline completo de processamento de refinamentos.
    
    Args:
        refinamentos (list): Lista de refinamentos
        poema_atual (str): Poema atual
    
    Returns:
        dict: Resultado do processamento
    """
    # 1. Validar cada refinamento e mapear alvos
    erros = []
    refinamentos_mapeados = []
    
    for i, ref in enumerate(refinamentos):
        validacao = validar_refinamento(ref, poema_atual)
        if not validacao['valido']:
            erros.append(f"Refinamento {i+1}: {validacao['erro']}")
        else:
            # Usar o alvo mapeado no refinamento
            ref_mapeado = ref.copy()
            ref_mapeado['alvo'] = validacao['alvo_mapeado']
            ref_mapeado['alvo_original'] = ref['alvo']  # Guardar o original para logs
            refinamentos_mapeados.append(ref_mapeado)
    
    if erros:
        return {'sucesso': False, 'erros': erros}
    
    # Usar os refinamentos mapeados daqui em diante
    refinamentos = refinamentos_mapeados
    
    # 2. Detectar conflitos
    conflitos = detectar_conflitos(refinamentos)
    if conflitos:
        avisos = [f"{c['descricao']}: {c['sugestao']}" for c in conflitos]
        # Continua mesmo com conflitos, mas avisa
    else:
        avisos = []
    
    # 3. Priorizar ordem
    refinamentos_ordenados = priorizar_refinamentos(refinamentos)
    
    # 4. Gerar prompts otimizados
    prompts = []
    for ref in refinamentos_ordenados:
        prompt = gerar_prompt_refinamento(ref, poema_atual)
        prompts.append({
            'refinamento': ref,
            'prompt': prompt
        })
    
    return {
        'sucesso': True,
        'avisos': avisos,
        'refinamentos_ordenados': refinamentos_ordenados,
        'prompts': prompts,
        'ordem_aplicacao': [r['acao'] for r in refinamentos_ordenados]
    }

print("Sistema Completo de Feedback (Grupo 3) carregado!")
print("Funções disponíveis: validar_refinamento, detectar_conflitos, priorizar_refinamentos")
print("Templates: gerar_prompt_refinamento, avaliar_refinamento, sugerir_refinamentos")
print("Pipeline principal: processar_refinamentos")

### FUNÇÕES DE GERAÇÃO INTEGRADAS (HISTÓRICO + FEEDBACK AVANÇADO)

In [ ]:
def gerar_poema_com_historico_avancado(tema, contexto, temperatura=0.7, usar_refinamentos=True, refinamentos=[]):
    """
    Gera um poema com histórico e sistema de feedback avançado integrado.
    
    Args:
        tema (str): Tema do poema
        contexto (str): Contexto/instruções adicionais
        temperatura (float): Temperatura para o modelo
        usar_refinamentos (bool): Se deve usar o sistema de refinamentos
        refinamentos (list): Lista de refinamentos a aplicar
    
    Returns:
        tuple: (poema_final, historico_detalhado, status)
    """
    global historico_poemas
    
    try:
        # 1. GERAÇÃO DO POEMA INICIAL
        prompt = f"""
        Crie um poema sobre o tema: {tema}
        
        Contexto adicional: {contexto}
        
        Instruções:
        - Seja criativo e expressivo
        - Use linguagem poética rica
        - Mantenha coerência temática
        - Estruture bem os versos e estrofes
        """
        
        response = model.generate_content(prompt)
        poema_inicial = response.text.strip()
        
        # Inicializar histórico para esta sessão
        versao_atual = 1
        historico_sessao = {
            'tema': tema,
            'contexto': contexto,
            'versoes': [
                {
                    'versao': versao_atual,
                    'poema': poema_inicial,
                    'timestamp': datetime.now().strftime("%H:%M:%S"),
                    'tipo': 'geração_inicial',
                    'refinamentos_aplicados': []
                }
            ]
        }
        
        poema_final = poema_inicial
        
        # 2. APLICAR SISTEMA DE REFINAMENTOS (SE SOLICITADO)
        if usar_refinamentos and refinamentos:
            print("Aplicando sistema de refinamentos avançado...")
            
            # Processar refinamentos usando o sistema do Grupo 3
            resultado_processamento = processar_refinamentos(refinamentos, poema_final)
            
            if not resultado_processamento['sucesso']:
                return poema_final, historico_sessao, f"❌ Erro nos refinamentos: {'; '.join(resultado_processamento['erros'])}"
            
            # Mostrar avisos se existirem
            if resultado_processamento['avisos']:
                print("⚠️ Avisos encontrados:")
                for aviso in resultado_processamento['avisos']:
                    print(f"   • {aviso}")
            
            # Aplicar cada refinamento sequencialmente
            for i, prompt_data in enumerate(resultado_processamento['prompts']):
                refinamento = prompt_data['refinamento']
                prompt_refinamento = prompt_data['prompt']
                
                print(f"   Aplicando refinamento {i+1}/{len(resultado_processamento['prompts'])}: {refinamento['acao']}")
                
                # Gerar versão refinada
                try:
                    response = model.generate_content(prompt_refinamento)
                    poema_refinado = response.text.strip()
                    
                    # Avaliar se o refinamento foi bem aplicado
                    avaliacao = avaliar_refinamento(poema_final, poema_refinado, refinamento)
                    
                    # Validação adicional: garantir que não é reescrita completa
                    similaridade = difflib.SequenceMatcher(None, poema_final.lower(), poema_refinado.lower()).ratio()
                    
                    if similaridade < 0.3:  # Muito diferente, provavelmente reescrita
                        print(f"   ⚠️ Refinamento {i+1} resultou em reescrita muito extensa (similaridade: {similaridade:.2f})")
                        print("   🔄 Tentando prompt mais conservador...")
                        
                        # Prompt de fallback mais conservador
                        prompt_conservador = f"""
                        POEMA ATUAL:
                        {poema_final}
                        
                        INSTRUÇÃO SIMPLES: Aplique apenas esta pequena modificação: {refinamento['acao']} "{refinamento['alvo']}" {f"para '{refinamento.get('novo_conteudo', '')}'" if refinamento.get('novo_conteudo') else ""}
                        
                        CRÍTICO: Retorne o poema com 95% do texto original preservado. Mude apenas o mínimo necessário.
                        """
                        
                        response_conservador = model.generate_content(prompt_conservador)
                        poema_refinado = response_conservador.text.strip()
                        avaliacao = avaliar_refinamento(poema_final, poema_refinado, refinamento)
                    
                    # Atualizar histórico
                    versao_atual += 1
                    historico_sessao['versoes'].append({
                        'versao': versao_atual,
                        'poema': poema_refinado,
                        'timestamp': datetime.now().strftime("%H:%M:%S"),
                        'tipo': 'refinamento',
                        'refinamento_aplicado': {
                            'acao': refinamento['acao'],
                            'alvo': refinamento.get('alvo_original', refinamento['alvo']),
                            'novo_conteudo': refinamento.get('novo_conteudo', ''),
                            'sucesso': avaliacao.get('sucesso', False),
                            'motivo': avaliacao.get('motivo', 'N/A')
                        }
                    })
                    
                    poema_final = poema_refinado
                    print(f"Refinamento aplicado - Sucesso: {avaliacao.get('sucesso', 'N/A')}")
                    
                except Exception as e:
                    print(f"Erro ao aplicar refinamento {i+1}: {str(e)}")
                    # Continua com o próximo refinamento
        
        # 3. SALVAR NO HISTÓRICO GLOBAL
        timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        entrada_historico = {
            'id': len(historico_poemas) + 1,
            'timestamp': timestamp,
            'tema': tema,
            'poema': poema_final,
            'contexto': contexto,
            'temperatura': temperatura,
            'versoes': len(historico_sessao['versoes']),
            'refinamentos_aplicados': usar_refinamentos and len(refinamentos) > 0,
            'historico_sessao': historico_sessao  # Histórico completo da sessão
        }
        
        historico_poemas.append(entrada_historico)
        
        # 4. PREPARAR RETORNO
        status = f"Poema gerado com sucesso! ({len(historico_sessao['versoes'])} versões criadas)"
        if usar_refinamentos and refinamentos:
            total_refinamentos = len(refinamentos)
            status += f" | {total_refinamentos} refinamentos aplicados"
        
        return poema_final, historico_sessao, status
        
    except Exception as e:
        return f"Erro ao gerar poema: {str(e)}", {}, "Erro na geração"

def aplicar_refinamentos_adicionais(poema_atual, refinamentos, tema=""):
    """
    Aplica refinamentos a um poema já existente (do histórico ou atual).
    
    Args:
        poema_atual (str): Poema a ser refinado
        refinamentos (list): Lista de refinamentos
        tema (str): Tema original (para contexto)
    
    Returns:
        tuple: (poema_refinado, detalhes_refinamento)
    """
    if not refinamentos:
        return poema_atual, "Nenhum refinamento fornecido"
    
    print(f"🔧 Aplicando {len(refinamentos)} refinamentos ao poema...")
    
    # Processar refinamentos
    resultado = processar_refinamentos(refinamentos, poema_atual)
    
    if not resultado['sucesso']:
        return poema_atual, f"Erro: {'; '.join(resultado['erros'])}"
    
    # Aplicar refinamentos sequencialmente
    poema_trabalhando = poema_atual
    detalhes = []
    
    for i, prompt_data in enumerate(resultado['prompts']):
        refinamento = prompt_data['refinamento']
        prompt_refinamento = prompt_data['prompt']
        
        try:
            print(f"   Refinamento {i+1}: {refinamento['acao']} - {refinamento.get('alvo_original', refinamento['alvo'])}")
            
            response = model.generate_content(prompt_refinamento)
            poema_novo = response.text.strip()
            
            # Avaliar refinamento
            avaliacao = avaliar_refinamento(poema_trabalhando, poema_novo, refinamento)
            
            detalhes.append({
                'refinamento': i+1,
                'acao': refinamento['acao'],
                'alvo': refinamento.get('alvo_original', refinamento['alvo']),
                'sucesso': avaliacao.get('sucesso', False),
                'motivo': avaliacao.get('motivo', 'N/A')
            })
            
            poema_trabalhando = poema_novo
            
        except Exception as e:
            detalhes.append({
                'refinamento': i+1,
                'acao': refinamento['acao'],
                'alvo': refinamento.get('alvo_original', refinamento['alvo']),
                'sucesso': False,
                'motivo': f"Erro: {str(e)}"
            })
    
    # Resumo
    sucessos = sum(1 for d in detalhes if d['sucesso'])
    resumo = f"{sucessos}/{len(detalhes)} refinamentos aplicados com sucesso"
    
    return poema_trabalhando, resumo

print("Funções de geração integradas com sistema de feedback avançado!")

### Configuração da API e Interface interativa com widgets

In [ ]:
import gradio as gr
import google.generativeai as genai
from datetime import datetime
import os

# --- Configuração da API do Gemini (Compatível com sistema avançado) ---

# Configurar a chave API para o notebook diretamente aqui
os.environ["GEMINI_API_KEY"] = "SUA_CHAVE_AQUI"

try:
    # Configurar API key do Google Gemini
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key or api_key == "SUA_CHAVE_AQUI":
        raise ValueError("GEMINI_API_KEY não configurada corretamente")
    
    genai.configure(api_key=api_key)
    
    # Criar modelo com configurações específicas
    model = genai.GenerativeModel(
        model_name="gemini-2.5-pro", 
        generation_config=genai.types.GenerationConfig(
            temperature=0.7,
        ),
        system_instruction="Você é um poeta de renome, especialista em vários estilos. Sua única saída deve ser o poema (refinado ou gerado) em português, sem explicações, saudações ou markdown adicional."
    )
    print("✅ API do Gemini configurada com sucesso!")
    print(f"🔗 Usando chave: {api_key[:10]}...")
except Exception as e:
    print(f"❌ Erro ao configurar a API do Gemini: {e}")
    print("Certifique-se de ter configurado sua GEMINI_API_KEY corretamente na célula anterior.")
    model = None

# --- Inicialização do histórico global ---
historico_poemas = []

# --- Funções Auxiliares para o Histórico ---
def formatar_historico_choices(historico):
    """Formata o histórico para ser usado no gr.Radio, mostrando o índice e as primeiras linhas."""
    choices = []
    if historico:
        for i, poema in enumerate(historico):
            primeiras_linhas = " ".join(poema.split('\n')[:2]).strip()
            if len(primeiras_linhas) > 60:
                primeiras_linhas = primeiras_linhas[:57] + "..."
                
            choices.append(f"V{i + 1}: {primeiras_linhas}")
            
    return choices


def selecionar_poema_historico(historico, radio_label_selecionado):
    """Atualiza o poema principal com base na seleção do histórico de rádio."""
    if not radio_label_selecionado or not historico:
        return historico[-1] if historico else "", historico[-1] if historico else ""

    try:
        versao_num = int(radio_label_selecionado.split(':')[0][1:])
        indice = versao_num - 1
        
        if 0 <= indice < len(historico):
            poema_selecionado = historico[indice]
            return poema_selecionado, poema_selecionado 
        else:
            return historico[-1], historico[-1]
    except Exception as e:
        print(f"Erro ao selecionar o poema do histórico: {e}")
        return historico[-1], historico[-1]


# --- Função de Backend para Geração/Refinamento (INTEGRADA COM SISTEMA AVANÇADO) ---
def gerar_poema_com_historico(tema, estilo, tamanho, temperatura, poema_atual, refinamentos_list, chat_atual, historico):
    """
    Função adaptada para usar o sistema avançado de feedback integrado.
    Mantém compatibilidade com a interface Gradio existente.
    """
    
    temp_normalizada = temperatura / 100.0
    
    # Verificar se tema está definido
    if tema == "":
        return f"❌ Defina um tema", poema_atual, gr.Tabs(selected=0), chat_atual, historico, gr.update(choices=formatar_historico_choices(historico), value=None)
    
    # Se há refinamentos, usar o sistema avançado
    if refinamentos_list and len(refinamentos_list) > 0:
        print(f"🔧 Usando sistema avançado para {len(refinamentos_list)} refinamentos")
        
        # Aplicar refinamentos usando o sistema do Grupo 3
        poema_refinado, detalhes = aplicar_refinamentos_adicionais(
            poema_atual, 
            refinamentos_list, 
            tema
        )
        
        # Adicionar ao histórico
        historico.append(poema_refinado)
        choices = formatar_historico_choices(historico)
        
        return poema_refinado, poema_refinado, gr.Tabs(selected=1), None, historico, gr.Radio(choices=choices, value=choices[-1] if choices else None)
    
    # Se não há refinamentos, gerar novo poema
    else:
        # Construir contexto baseado no estilo e tamanho
        contexto = f"Estilo: {estilo}. "
        if tamanho == "Curto (1-2 estrofes)":
            contexto += "Poema deve ter 1 ou 2 estrofes."
        elif tamanho == "Médio (3-4 estrofes)":
            contexto += "Poema deve ter 3 ou 4 estrofes."
        else:
            contexto += "Poema deve ter 5 ou mais estrofes."
        
        try:
            # Usar sistema avançado de geração (sem refinamentos nesta primeira etapa)
            poema_final, historico_sessao, status = gerar_poema_com_historico_avancado(
                tema=tema,
                contexto=contexto,
                temperatura=temp_normalizada,
                usar_refinamentos=False,  # Primeira geração sem refinamentos
                refinamentos=[]
            )
            
            # Resetar histórico para novo tema (preservar tema anterior)
            if tema != "" and (not historico or tema not in str(historico)):
                historico = []
            
            # Adicionar ao histórico
            historico.append(poema_final)
            choices = formatar_historico_choices(historico)
            
            return poema_final, poema_final, gr.Tabs(selected=1), None, historico, gr.Radio(choices=choices, value=choices[-1] if choices else None)
            
        except Exception as e:
            return f"❌ Erro ao gerar poema: {str(e)}", poema_atual, gr.Tabs(selected=0), chat_atual, historico, gr.update(choices=formatar_historico_choices(historico), value=None)




# --- Funções de Controle ---
def limpar_campos():
    """Limpa campos e estados, incluindo o histórico."""
    return (
        "", 
        "Livre (sem estrutura específica)", 
        "Médio (3-4 estrofes)", 
        70, 
        "", 
        [], 
        None, 
        [], 
        gr.update(choices=[], value=None)
    )

def toggle_refinamentos():
    return gr.Column(visible=True)

def adicionar_refinamento(refinamentos, acao, alvo, novo_conteudo):
    if not alvo:
        return refinamentos, formatar_refinamentos(refinamentos), "Editar", "", "" 

    refinamentos.append({
        'acao': acao,
        'alvo': alvo,
        'novo_conteudo': novo_conteudo
    })

    return refinamentos, formatar_refinamentos(refinamentos), "Editar", "", ""

def limpar_refinamentos():
    return [], "*Nenhum refinamento adicionado ainda*"

def formatar_refinamentos(refinamentos):
    if not refinamentos:
        return "*Nenhum refinamento adicionado ainda*"

    texto = ""
    for i, ref in enumerate(refinamentos, 1):
        texto += f"**{i}. {ref['acao']}**: {ref['alvo']}"
        if ref['novo_conteudo']:
            texto += f" → *{ref['novo_conteudo']}*"
        texto += "\n\n"
    return texto

def voltar_criar():
    return gr.Tabs(selected=0)

# --- FUNÇÃO PARA CONTROLAR A VISIBILIDADE DO CAMPO NOVO CONTEÚDO ---
def toggle_novo_conteudo(acao):
    """
    Controla se o campo 'Novo Conteúdo' deve ser visível ou editável 
    baseado na ação de refinamento (Adicionar/Editar/Remover).
    """
    if acao == "Adicionar" or acao == "Editar":
        # Necessita de novo conteúdo. Tornar visível e interativo.
        return gr.update(visible=True, interactive=True)
    else: # acao == "Remover"
        # Não precisa de novo conteúdo. Tornar invisível.
        return gr.update(visible=False, interactive=False)


# --- CSS Personalizado (AJUSTADO PARA LARGURA E ALTURA) ---
custom_css = """
.gradio-container {
    max-width: 1200px !important;
    margin: auto !important;
}
h1 {
    text-align: center;
    color: #2c3e50;
    font-size: 2.5em;
    margin-bottom: 10px;
}
.subtitle {
    text-align: center;
    color: #7f8c8d;
    font-size: 1.2em;
    margin-bottom: 30px;
}
.generate-btn {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    border: none !important;
    color: white !important;
    font-size: 1.1em !important;
    padding: 15px 30px !important;
    border-radius: 10px !important;
    font-weight: bold !important;
    transition: transform 0.2s !important;
}
.generate-btn:hover {
    transform: scale(1.05) !important;
}
.limpar-btn {
    background: #95a5a6 !important;
    color: white !important;
    border-radius: 8px !important;
}
.button-row {
    display: flex !important;
    justify-content: space-between !important;
    gap: 10px !important;
}
.refinamento-box {
    border: 2px solid #667eea;
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    background: #f8f9fa;
    /* Garante que o box de refinamento ocupe a largura completa da coluna */
    width: 100%; 
}
.history-container {
    padding: 10px;
    border: 1px solid #ddd;
    border-radius: 10px;
    background: #f0f4f7;
    height: 100%;
    min-height: 400px;
}
.history-radio > label {
    border: 1px solid #bdc3c7;
    border-radius: 8px;
    margin-bottom: 5px;
    padding: 8px;
    display: block;
    cursor: pointer;
    transition: background-color 0.2s;
    background-color: white;
}
.history-radio > label:hover {
    background-color: #ecf0f1;
}
.history-radio .gr-check-radio-label {
    padding: 0;
    margin: 0;
}
.history-radio input[type="radio"]:checked + .gr-check-radio-label {
    background-color: #e8e6f0;
    border-color: #667eea;
    font-weight: bold;
}
.history-radio input[type="radio"] {
    display: none;
}
/* CLASSE PARA FIXAR ALTURA E EVITAR SALTO VERTICAL */
.poema-refinamento-fixo {
    min-height: 800px; 
}
/* Garante que o layout da coluna do poema/refinamento use 100% da largura alocada */
.column.poema-refinamento-fixo {
    box-sizing: border-box;
}
"""


# --- Interface Gradio ---
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="Gerador de Poemas IA") as demo:
    # --- Estados ---
    poema_state = gr.State(value="")
    refinamentos_state = gr.State(value=[])
    chat_state = gr.State(value=None)
    historico_state = gr.State(value=[])

    gr.Markdown("# Gerador de Poemas com IA")
    gr.Markdown("<p class='subtitle'>✨ Transforme suas ideias em versos únicos e inspiradores</p>")

    with gr.Tabs() as tabs:
        # ABA 1: Definição do Tema
        with gr.Tab("📝 Criar Poema", id=0):
            gr.Markdown("### Defina o tema e estilo do seu poema")

            with gr.Row():
                with gr.Column(scale=2):
                    tema_input = gr.Textbox(
                        label="Tema do Poema",
                        placeholder="Ex: A beleza do pôr do sol, saudade, amor impossível...",
                        lines=3,
                        max_lines=5
                    )

                    estilo_input = gr.Dropdown(
                        label="Estilo do Poema",
                        choices=[
                            "Livre (sem estrutura específica)",
                            "Romântico",
                            "Haicai (estilo japonês)",
                            "Soneto",
                            "Moderno",
                            "Clássico",
                            "Melancólico",
                            "Alegre e descontraído"
                        ],
                        value="Livre (sem estrutura específica)"
                    )

                    tamanho_input = gr.Radio(
                        label="Tamanho do Poema",
                        choices=["Curto (1-2 estrofes)", "Médio (3-4 estrofes)", "Longo (5+ estrofes)"],
                        value="Médio (3-4 estrofes)"
                    )

                with gr.Column(scale=1):
                    gr.Markdown("### ⚙️ Configurações Avançadas")

                    temperatura_input = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=70,
                        step=1,
                        label="Criatividade",
                        info="Maior = mais criativo e variado"
                    )

                    gr.Markdown("---")
                    gr.Markdown("### 💡 Exemplos de Temas")
                    gr.Markdown("""
                    - O som da chuva à noite
                    - Um gato observando a lua
                    - A solidão na cidade grande
                    - Memórias de infância
                    """)

            with gr.Row(elem_classes=["button-row"]):
                gerar_btn = gr.Button("🚀 Gerar Poema", variant="primary", elem_classes=["generate-btn"])
                limpar_btn = gr.Button("🗑️ Limpar", variant="secondary", elem_classes=["limpar-btn", "generate-btn"])

        # ABA 2: Resultado do Poema
        with gr.Tab("📜 Seu Poema", id=1):
            with gr.Row():
                # COLUNA 1: Poema Atual e Refinamentos
                with gr.Column(scale=4, elem_classes=["poema-refinamento-fixo"]):
                    gr.Markdown("### Poema")
                    
                    poema_output = gr.Textbox(
                        label="",
                        lines=15,
                        max_lines=20,
                        show_label=False,
                        interactive=False,
                        placeholder="Seu poema aparecerá aqui após a geração..."
                    )
                    
                    gr.Markdown("---")

                    # Botão para mostrar/ocultar refinamentos
                    with gr.Row():
                        mostrar_refinamentos_btn = gr.Button("➕ Adicionar Refinamentos", variant="secondary")

                    # Container de refinamentos (inicialmente oculto)
                    refinamentos_container = gr.Column(visible=False)

                    with refinamentos_container:
                        gr.Markdown("### ✂️ Refinamentos")

                        # Container dinâmico para múltiplos blocos
                        refinamentos_accordion = gr.Accordion("Refinamentos Adicionados", open=True)

                        with refinamentos_accordion:
                            refinamentos_display = gr.Markdown("*Nenhum refinamento adicionado ainda*")

                        gr.Markdown("#### Adicionar Novo Refinamento")

                        # GRUPO DE ENTRADA DO REFINAMENTO
                        with gr.Group(elem_classes=["refinamento-box"]):
                            with gr.Column():
                                acao_temp = gr.Radio(
                                    label="Ação",
                                    choices=["Adicionar", "Editar", "Remover"],
                                    value="Editar"
                                )
                                alvo_temp = gr.Textbox(
                                    label="Trecho/Alvo",
                                    placeholder="Ex: 'A terceira estrofe', 'A palavra vento'",
                                    lines=1
                                )
                                # O CAMPO NOVO CONTEÚDO SERÁ CONTROLADO POR JS
                                novo_conteudo_temp = gr.Textbox(
                                    label="Novo Conteúdo",
                                    placeholder="Digite o novo texto (visível para Adicionar/Editar)",
                                    lines=2,
                                    visible=True, # Inicia visível, mas a lógica no .change corrige
                                    interactive=True
                                )

                        with gr.Row():
                            adicionar_refinamento_btn = gr.Button("➕ Adicionar à Lista", variant="secondary")
                            limpar_refinamentos_btn = gr.Button("🗑️ Limpar Todos", variant="secondary")

                        refinar_btn = gr.Button("🔄 Aplicar Refinamentos", variant="primary", elem_classes=["generate-btn"])

                    # LINHA DE BOTÕES FINAL
                    with gr.Row():
                        # O botão Criar Novo Poema ocupa todo o espaço da linha
                        novo_btn = gr.Button("✏️ Criar Novo Poema", variant="primary", size="lg") 

                # COLUNA 2: Histórico
                with gr.Column(scale=1, elem_classes=["history-container"]):
                    gr.Markdown("### 📜 Histórico")
                    gr.Markdown("Selecione a versão para torná-la a atual e aplicar refinamentos.")
                    
                    historico_radio = gr.Radio(
                        label="",
                        choices=[],
                        value=None,
                        show_label=False,
                        elem_classes=["history-radio"]
                    )


    # --- Conectar Eventos ---
    
    # CONEXÃO DA LÓGICA DE VISIBILIDADE DO CAMPO NOVO CONTEÚDO
    acao_temp.change(
        fn=toggle_novo_conteudo,
        inputs=[acao_temp],
        outputs=[novo_conteudo_temp]
    )

    # 1. Geração inicial
    gerar_btn.click(
        fn=lambda t, e, tam, temp, p, r, c, h: gerar_poema_com_historico(t, e, tam, temp, p, [], c, h),
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state, historico_state],
        outputs=[poema_output, poema_state, tabs, chat_state, historico_state, historico_radio]
    )

    # 2. Toggle refinamentos
    mostrar_refinamentos_btn.click(
        fn=toggle_refinamentos,
        outputs=[refinamentos_container]
    )

    # 3. Adicionar refinamento à lista
    adicionar_refinamento_btn.click(
        fn=adicionar_refinamento,
        inputs=[refinamentos_state, acao_temp, alvo_temp, novo_conteudo_temp],
        outputs=[refinamentos_state, refinamentos_display, acao_temp, alvo_temp, novo_conteudo_temp]
    )

    # 4. Limpar refinamentos
    limpar_refinamentos_btn.click(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )

    # 5. Aplicar refinamentos
    refinar_btn.click(
        fn=gerar_poema_com_historico,
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state, historico_state],
        outputs=[poema_output, poema_state, tabs, chat_state, historico_state, historico_radio]
    ).then(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )
    
    # 6. Selecionar poema do histórico (Atualiza o poema principal)
    historico_radio.change(
        fn=selecionar_poema_historico,
        inputs=[historico_state, historico_radio],
        outputs=[poema_output, poema_state]
    )

    # 7. Outros botões
    limpar_btn.click(fn=limpar_campos, outputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state,
                                                refinamentos_state, chat_state, historico_state, historico_radio])
    novo_btn.click(fn=voltar_criar, outputs=tabs)

# --- Lançar Interface ---
demo.launch(inline=True, share=True)